In [3]:
import time
import torch
import cv2
import numpy as np
from torchvision import transforms

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

ModuleNotFoundError: No module named 'utils'

In [4]:
!pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/13/24/23cdf7e7dc33e5c01588c315f8424d31afa9edb05a80168f3d44f7178ff7/torchvision-0.16.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------------------- ------------------ 0.6/1.1 MB 12.6 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 14.5 MB/s eta 0:00:00


In [ ]:
def pose_video(frame):
    # 현재 프레임을 복사합니다.
    mapped_img = frame.copy()
    # 프레임을 입력 사이즈에 맞게 재조정합니다.
    img = letterbox(frame, input_size, stride=64, auto=True)[0]
    print(img.shape)
    img_ = img.copy()
    # 배열을 4D 텐서로 변환합니다.
    img = transforms.ToTensor()(img)
    # 텐서로 변환된 배열을 다시 NumPy 배열로 변환합니다.
    img = torch.tensor(np.array([img.numpy()]))
    # 이미지를 계산 장치(CPU 또는 GPU)로 로드합니다.
    img = img.to(device)
    
    # 추론 시에는 그래디언트를 저장하지 않습니다.
    with torch.no_grad():
        t1 = time.time()
        output, _ = model(img)
        t2 = time.time()
        fps = 1/(t2 - t1)
        # 비최대 억제를 사용하여 출력을 처리합니다.
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=1, nkpt=17, kpt_label=True)
        # 출력을 키포인트로 변환합니다.
        output = output_to_keypoint(output)

    # 이미지 형식을 [b, c, h, w]에서 [h, w, c]로 변경하여 화면에 표시합니다.
    nimg = img[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    # 출력된 각 객체에 대한 포즈를 그립니다.
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
        
    return nimg, fps

#------------------------------------------------------------------------------#

# 포워드 패스의 입력 크기를 설정합니다.
input_size = 256

#---------------------------초기화---------------------------------------------#

# 하드웨어 구성에 따라 장치를 선택합니다.
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print('Selected Device : ', device)

# 키포인트 감지 모델을 로드합니다.
weights = torch.load('yolov7-w6-pose.pt', map_location=device)
model = weights['model']
# 모델을 평가 모드로 설정합니다.
_ = model.float().eval()
# 모델을 계산 장치에 로드합니다.
model.to(device)

# 비디오 캡쳐와 라이터를 초기화합니다.
videos = ['dance', 'dark', 'far-away', 'occlusion-example', 'skydiving', 'yoga-1']

file_name = videos[0] + '.mp4'
vid_path = '../Media/' + file_name

cap = cv2.VideoCapture(vid_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
ret, frame = cap.read()
h, w, _ = frame.shape

# letterbox 함수가 이미지 크기를 변경하므로 w, h를 변경해야 할 수 있습니다.
out = cv2.VideoWriter('pose_outputs/' + file_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
#-------------------------------------------------------------------------------#

if __name__ == '__main__':
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print('Unable to read frame. Exiting ..')
            break

        img, fps_ = pose_video(frame)

        # FPS와 모델명을 화면에 표시합니다.
        cv2.putText(img, 'FPS : {:.2f}'.format(fps_), (200, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
        cv2.putText(img, 'YOLOv7', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)

        # 결과를 화면에 표시하고 저장합니다.
        cv2.imshow('Output', img[...,::-1])
        out.write(img[...,::-1])
        key = cv2.waitKey(1)
        if key == ord('q'):
        	break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
